<a href="https://colab.research.google.com/github/poomshift/WebUI-Forge-Colab-Alchemist/blob/main/WebUI_Forge_Colab_Alchemist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@markdown #1. Install WebUI Forge | Models (2-3 นาที)

from IPython.display import clear_output
from IPython.display import Javascript
import ipywidgets as widgets
from google.colab import drive
import os
from google.colab import output
import subprocess
import sys


output.enable_custom_widget_manager()

display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)

%cd /content
drive.mount('/content/drive')



#ติดตั้ง WebUI Forge
!git clone https://github.com/lllyasviel/stable-diffusion-webui-forge

#ติดตั้ง Dependencies
%cd /content/stable-diffusion-webui-forge
!apt -y install -qq aria2
!pip install xformers!=0.0.18 torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121
# !pip install -r requirements_versions.txt

def install_requirements(requirements_file):
    # Install packages without asking for restart
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', '-r', requirements_file])

# Usage
install_requirements('requirements_versions.txt')

#Extension
!git clone https://github.com/zanllp/sd-webui-infinite-image-browsing /content/stable-diffusion-webui-forge/extensions/sd-webui-infinite-image-browsing
!git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111 /content/stable-diffusion-webui-forge/extensions/ultimateSD
!git clone https://github.com/BlafKing/sd-civitai-browser-plus /content/stable-diffusion-webui-forge/extensions/sd-civitai-browser-plus

%cd /content/stable-diffusion-webui-forge/extensions/sd-webui-infinite-image-browsing
!pip install -r requirements.txt

%cd /content/stable-diffusion-webui-forge

# Define the file path and content
file_path = '/content/stable-diffusion-webui-forge/config.json'
config_content = '''
{
  "outdir_samples": "",
  "outdir_txt2img_samples": "/content/drive/MyDrive/Forge_output/text",
  "outdir_img2img_samples": "/content/drive/MyDrive/Forge_output/image",
  "outdir_extras_samples": "/content/drive/MyDrive/Forge_output/extras",
  "outdir_grids": "",
  "outdir_txt2img_grids": "/content/drive/MyDrive/Forge_output/grids",
  "outdir_img2img_grids": "/content/drive/MyDrive/Forge_output/grids",
  "outdir_save": "/content/drive/MyDrive/Forge_output/save",
  "outdir_init_images": "/content/drive/MyDrive/Forge_output/init-images"
}
'''
# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# Write the config file
with open(file_path, 'w') as config_file:
    config_file.write(config_content)


#Flux VAE, Clip
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -c https://huggingface.co/black-forest-labs/FLUX.1-schnell/resolve/main/ae.safetensors -d /content/stable-diffusion-webui-forge/models/VAE -o ae.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -c https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors -d /content/stable-diffusion-webui-forge/models/text_encoder -o clip_l.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -c https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp8_e4m3fn.safetensors -d /content/stable-diffusion-webui-forge/models/text_encoder -o t5xxl_fp8_e4m3fn.safetensors
#@markdown Flux1Dev
shuttle_3_diffusion_Q4_gguf_4_steps = True # @param {type:"boolean"}
if shuttle_3_diffusion_Q4_gguf_4_steps == True:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -c https://huggingface.co/shuttleai/shuttle-3-diffusion-GGUF/resolve/main/shuttle-3-diffusion-Q4_K_S.gguf -d /content/stable-diffusion-webui-forge/models/Stable-diffusion -o shuttle-3-diffusion-Q4_K_S.gguf
else:
  ()

#cloudflare tunnel
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

inf('\u2714 Complete','success', '50px')

<IPython.core.display.Javascript object>

/content
Mounted at /content/drive
Cloning into 'stable-diffusion-webui-forge'...
remote: Enumerating objects: 41718, done.
remote: Counting objects: 100% (6760/6760), done.
remote: Compressing objects: 100% (532/532), done.
remote: Total 41718 (delta 6290), reused 6234 (delta 6228), pack-reused 34958 (from 1)
Receiving objects: 100% (41718/41718), 47.89 MiB | 44.99 MiB/s, done.
Resolving deltas: 100% (28713/28713), done.
/content/stable-diffusion-webui-forge
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 123630 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking l

Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

In [ ]:
#@markdown #2. Run WebUI Forge
%cd /content/stable-diffusion-webui-forge
import subprocess
import threading
import time
import socket
from IPython.display import clear_output
from IPython.display import Javascript
import ipywidgets as widgets

# Set up display
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))


# Launch WebUI with Gradio share
!python launch.py --listen --share --port 7860 --api

/content/stable-diffusion-webui-forge


<IPython.core.display.Javascript object>

Python 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
Version: f2.0.1v1.10.1-previous-622-g1f140749
Commit hash: 1f140749d4ccb4136d7af3bf546c21688e878df6
Legacy Preprocessor init warning: Unable to install insightface automatically. Please try run `pip install insightface` manually.
Launching Web UI with arguments: --listen --share --port 7860 --api
Total VRAM 15102 MB, total RAM 52217 MB
pytorch version: 2.5.1+cu121
xformers version: 0.0.28.post3
Set vram state to: NORMAL_VRAM
Device: cuda:0 Tesla T4 : native
VAE dtype preferences: [torch.float32] -> torch.float32
CUDA Using Stream: False
2024-11-22 07:18:16.146631: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-22 07:18:16.166565: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has al